In [ ]:
import os
import pdb
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from matplotlib import pyplot as plt
import decoupler as dc
import scanpyHelpers as scH
from collections import Counter
from sklearn.mixture import BayesianGaussianMixture as GMM
from scipy.stats import norm
import scipy.stats as stats


In [ ]:
writeDir="write/"

In [ ]:
for file in os.listdir(writeDir):
    if "luad" in file.lower():
        print(f"\'{file}\',")

In [ ]:
luadFiles = [#'luadPdx.h5ad',
#'LUADpdxLX55A.h5ad',
#'LUADpdxJHU55A.h5ad',
#'LUADpdxRu890.h5ad',
#'LUADpdxRu465.h5ad',
#'LUADpdxLX369.h5ad',
###'LUADpdxLX29.h5ad',
#'LUADpdxLX278.h5ad',
###'LUADpdxLX29A.h5ad',
#'LUADpdxRu699.h5ad',
#'LUADpdxRu318.h5ad',
#'LUADpdxRu210.h5ad',
#'luadBischoff.h5ad',
    
'luadBischoffp018.h5ad',
'luadBischoffp032.h5ad',
'luadBischoffp024.h5ad',
#'luadKim.h5ad',
'luadKimP0034.h5ad',
'luadKimP0030.h5ad',
'luadDost.h5ad',
'LUAD_P2_Malig_re.h5ad',
'LUAD_P3_Malig_re.h5ad',
'LUAD_P5_Malig_re.h5ad'
#'luadLaughney.h5ad'
]

#'luadMouse_shKras.h5ad',


In [ ]:
sigFile = "data/LUAD/humanLUADgeneSig.csv"
sigGenes = pd.read_csv(sigFile)
sigGenes

markers = pd.DataFrame(np.empty((500*12,2), dtype="object"), columns=["genes", "clustName"])
for i,clust in enumerate(sigGenes.columns):
    for j,gene in enumerate(sigGenes.index):
        gene = sigGenes.loc[gene,clust]
        #indexOver = np.where(mouseGenes == gene)[0]
        #if indexOver.size > 0:
        #    gene = humanGenes[indexOver[0]]
        markers.iloc[i*500+j] = gene,clust
        
markers = markers.dropna()
markers

In [ ]:
nfkb = pd.read_table("data/LUAD/hallmarkTnfaNFKB.csv", sep=",").columns.values
kac = pd.read_table("data/LUAD/mdaKAC.txt", header=None)[0].values#[:200]

In [ ]:
set(sigGenes["HPCS"]).intersection(set(kac))

In [ ]:
luadf="luadBischoffp032.h5ad"
adata = sc.read(writeDir+luadf)
sc.tl.leiden(adata, resolution=0.3)
scoreNames, newBClabel, ogLabelScoreMed = scH.scoreAndLabel(adata, sigGenes, labelOfSigGenes= sigGenes.columns, ogLabel="leiden", translate=False)
sc.tl.score_genes(adata, nfkb,score_name='nfkbScore')
sc.tl.score_genes(adata, kac,score_name='kacScore')


In [ ]:
sc.pl.umap(adata, color=scoreNames+["cellState","leiden","nfkbScore","kacScore"], ncols=4, cmap="bwr")

In [ ]:
from sklearn.mixture import BayesianGaussianMixture as GMM
#from sklearn.mixture import GaussianMixture as GMM

def gmmScoreGeneSig(scoreMat, meansInit=[[0.1],[0.4]],plotLen = 3, show=False):
    scoreNames = scoreMat.columns
    numScores = len(scoreNames)
    if(show):
        fig, axs = plt.subplots((numScores//plotLen)+1,plotLen)
        plt.rcParams["figure.figsize"] = (15,5)

    dfScoreBoundry = pd.DataFrame(np.zeros(numScores),scoreNames, columns=["boundry"])
    gmm = GMM(n_components = 2, random_state=10)#, init_params="random_from_data")#, means_init=meansInit)
    #binEx = np.arange(0.5,10,10/200).reshape(-1,1)

    for i, scoreName in enumerate(scoreNames):
        scoreCount = np.array(scoreMat[scoreName]).reshape(-1, 1)
        fitGMM = gmm.fit(scoreCount)
        mean = fitGMM.means_  
        covs  = fitGMM.covariances_
        weights = fitGMM.weights_
        #binEx = np.arange(min(min(mean),max(mean)),max(scoreCount),0.01).reshape(-1,1)
        binEx = np.arange(min(min(mean),max(mean))[0],max(scoreCount)[0],0.01).reshape(-1,1)
        fitGmmBound = fitGMM.predict(binEx)
        furtherBound = fitGmmBound[-1]
        #print(furtherBound)
        #print(fitGmmBound)
        if (len(np.unique(fitGmmBound)) == 2):
            scoreBoundry = binEx[np.where(fitGmmBound == furtherBound)[0][0]][0]
        else:
            scoreBoundry = max(scoreCount)
        #naiveBoundry = np.log10(int(scoreDisc.loc["90%",scoreName])+1)

        dfScoreBoundry.loc[scoreName] = scoreBoundry
        
        if(show):
            x_axis = np.arange(min(scoreCount), max(scoreCount), 0.01)
            y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
            y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian

            # Plot 2
            x,y = i//plotLen, i%plotLen
            axs[x,y].set_title(scoreName)
            #axs[x,y].axvline(naiveBoundry, c='C3', linestyle='dashed', linewidth=1) #red
            axs[x,y].axvline(scoreBoundry, c='C2', linestyle='dashed', linewidth=1)  #green
            axs[x,y].hist(scoreCount, density=True, color='black', bins=100)        
            axs[x,y].plot(x_axis, y_axis0, lw=3, c='C6')                            #pink
            axs[x,y].plot(x_axis, y_axis1, lw=3, c='C1')                            #orange
            axs[x,y].plot(x_axis, y_axis0+y_axis1, lw=3, c='C0', ls=':')            #dotted blue

    if(show):
        plt.tight_layout(pad=1.0)
        plt.show()
        #sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
    
    scoreIDs = scoreMat.copy()
    scoreID = np.array(scoreNames)
    #pdb.set_trace()

    for scoreName in scoreNames:
        #print(scoreName)
        #print(dfScoreBoundry.loc[scoreName].values[0])
        #if(dfScoreBoundry.loc[scoreName].values[0] > 0):
        scoreIDs.loc[:,scoreName] = (scoreMat.loc[:,scoreName] > dfScoreBoundry.loc[scoreName].values[0]).astype("float")

        
    classification = np.empty(len(scoreMat), dtype="object")
    i = 0
    for cellBar, scoreBool in scoreIDs.iterrows():
        scoreBool = scoreBool.astype("bool")
        numscorees = sum(scoreBool)
        if (numscorees == 1):
            classif = scoreID[scoreBool.values][0][:-5]#.values
        elif (numscorees > 1):
            #pdb.set_trace()
            #classif = "Doublet"
            maxTrue = np.argmax(adata.obs.loc[cellBar,scoreID[scoreBool]])   
            #pdb.set_trace()
            classif = scoreID[scoreBool][maxTrue][:-5]#.values
        else:
            classif = "Negative"
        classification[i] = classif
        i = i + 1
        
    return(classification)

In [ ]:
gmmClass = gmmScoreGeneSig(adata.obs[scoreNames],show=True)

In [ ]:
adata.obs["gmmClass"] = gmmClass

In [ ]:
Counter(gmmClass)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
sc.pl.umap(adata, color=scoreNames+["cellState","leiden","nfkbScore","kacScore","gmmClass"], ncols=4, cmap="bwr")

In [ ]:
sc.pl.umap(adata, color=["at2Score","hpcsScore","nfkbScore","gmmClass"], ncols=2, cmap="bwr")

In [ ]:

hpcsNFKb = adata[adata.obs.gmmClass=="HPCS"].obs["nfkbScore"]
otherNFKb = adata[np.logical_not(adata.obs.gmmClass=="HPCS")].obs["nfkbScore"]

# Check variance (if ratio > 4:1, variances are different)
variance_ratio = np.var(hpcsNFKb) / np.var(otherNFKb)
print(f"Variance ratio: {variance_ratio:.4f}")

# Conduct Welch’s t-test
t_statistic, p_value = stats.ttest_ind(hpcsNFKb, otherNFKb, equal_var=False, alternative="greater")

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The means are significantly different.")
else:
    print("Fail to reject the null hypothesis: No significant difference detected.")

print(f"Welch’s t-statistic: {t_statistic:.4f}, p-value: {p_value:.4f}")


u_statistic, p_value = stats.mannwhitneyu(hpcsNFKb, otherNFKb, alternative="greater")
if p_value < alpha:
    print("Reject the null hypothesis: The means are significantly different.")
else:
    print("Fail to reject the null hypothesis: No significant difference detected.")

print(f"mannwhitneyu’s u-statistic: {t_statistic:.4f}, p-value: {p_value:.4f}")

# Create a 1x2 subplot grid
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot histogram for Scores 1
axs[0].violinplot(hpcsNFKb,showmedians=True)#, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
axs[0].set_title('hpcsNFKb')
axs[0].set_xlabel('Score Range')
axs[0].set_ylabel('Frequency')

# Plot histogram for Scores 2
axs[1].violinplot(otherNFKb,showmedians=True)#, bins=20, color='salmon', edgecolor='black', alpha=0.7)
axs[1].set_title('otherNFKb')
axs[1].set_xlabel('Score Range')
axs[1].set_ylabel('Frequency')

# Plot histogram for Scores 2
axs[2].violinplot([hpcsNFKb, otherNFKb],showmedians=True)#, bins=20, color=['skyblue', 'salmon'], edgecolor='black', alpha=0.7, label=['hpcsNFKb', 'otherNFKb'])
axs[2].set_xticks([1, 2])
axs[2].set_xticklabels(['hpcsNFKb', 'otherNFKb'])
axs[2].set_xlabel('Score Range')
axs[2].set_ylabel('Frequency')
axs[2].legend()


# Add overall title
plt.suptitle('Histograms of Two Sets of Scores')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data for demonstration purposes
dataset_a = np.random.normal(loc=0, scale=1, size=100)
dataset_b = np.random.normal(loc=1, scale=2, size=100)

# Create a figure and a set of subplots
fig, ax = plt.subplots()

# Create the violin plot
violins = ax.violinplot([dataset_a, dataset_b], positions=[1, 1], showmeans=False, showmedians=False)

# Adjust the violins to only show the right side for dataset_b and left side for dataset_a
for body in violins['bodies']:
    m = np.mean(body.get_paths()[0].vertices[:, 0])
    if body == violins['bodies'][0]:  # dataset_a
        body.get_paths()[0].vertices[:, 0] = np.clip(body.get_paths()[0].vertices[:, 0], -np.inf, m)
    else:  # dataset_b
        body.get_paths()[0].vertices[:, 0] = np.clip(body.get_paths()[0].vertices[:, 0], m, np.inf)

# Set the x-ticks to correspond to the datasets
ax.set_xticks([1])
ax.set_xticklabels(['Dataset A & B'])

# Adding titles and labels
ax.set_title('Half-Sided Violin Plot of Dataset A and B')
ax.set_ylabel('Value')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate some example data
np.random.seed(10)
dataset_a = np.random.normal(loc=0, scale=1, size=100)
dataset_b = np.random.normal(loc=2, scale=1.5, size=100)

# Create a violin plot
plt.figure(figsize=(8, 6))
parts = plt.violinplot([dataset_a, dataset_b], showmeans=False, showmedians=True)

# Customize labels and title
plt.xticks([1, 2], ['Dataset A', 'Dataset B'])
plt.ylabel('Values')
plt.title('Violin Plot of Dataset A and Dataset B')

# Adjusting colors for the violins
colors = ['skyblue', 'salmon']
for pc, color in zip(parts['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_edgecolor('black')

# Show the plot
plt.grid(True)
plt.show()


In [ ]:
adata.obs["hpcs"] = ["HPCS" if h else "other" for h in adata.obs.gmmClass=="HPCS"]

In [ ]:
adata.obs

In [ ]:
sc.pl.violin(adata, keys="nfkbScore", groupby="hpcs",  inner = "box", stripplot=False,multi_panel=False)

In [ ]:
kac

In [ ]:
hpcsMarkers = []
hpcsLogfcs = []

allHpcsList = np.array([])

for luadf in luadFiles:
    print(luadf)
    adata = sc.read_h5ad(writeDir+luadf)
    sc.tl.leiden(adata, resolution=0.3)
    scoreNames, newBClabel, ogLabelScoreMed = scH.scoreAndLabel(adata, sigGenes, labelOfSigGenes= sigGenes.columns, ogLabel="leiden", translate=False)
    sc.tl.score_genes(adata, nfkb,score_name='nfkbScore')
    sc.tl.score_genes(adata, kac,score_name='kacScore')
    
    dc.run_ora(
    mat=adata,
    net=markers,
    source='clustName',
    target='genes',
    min_n=3,
    verbose=False)
    acts = dc.get_acts(adata, obsm_key='ora_estimate')

    # We need to remove inf and set them to the maximum value observed for pvals=0
    acts_v = acts.X.ravel()
    max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
    acts.X[~np.isfinite(acts.X)] = max_e
    
    df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
    annotation_dict = df.groupby('group').head(1).set_index('group')['names'].to_dict()
    adata.obs['decoupler'] = [annotation_dict[clust] for clust in adata.obs['leiden']]
    
    #try:
    #    sc.tl.rank_genes_groups(adata, 'decoupler', method='wilcoxon', use_raw=False)
    #except:
    #    adata.uns['log1p']['base'] = None
    #    sc.tl.rank_genes_groups(adata, 'decoupler', method='wilcoxon', use_raw=False)
    
    #only significant pVALS
    ##degs = pd.DataFrame({group + ‘_’ + key: result[key][group] for group in groups for key in [‘names’,‘scores’, ‘pvals_adj’,‘logfoldchanges’]})

    #sc.tl.filter_rank_genes_groups(adata, groupby="decoupler", use_raw=False,
    #                                   key_added='rank_genes_groups_filtered', 
    #                                   min_in_group_fraction=0.5, min_fold_change=0.5, max_out_group_fraction=0.5, compare_abs=False)
    #print(pd.DataFrame(adata.uns['rank_genes_groups_filtered']['names']).head(15))
    
    gmmClass = gmmScoreGeneSig(adata.obs[scoreNames],show=False)
    adata.obs["gmmClass"] = gmmClass

    #pdb.set_trace()
    
    try:
        sc.tl.rank_genes_groups(adata, 'gmmClass', method='wilcoxon', use_raw=True)
    except:
        adata.uns['log1p']['base'] = None
        sc.tl.rank_genes_groups(adata, 'gmmClass', method='wilcoxon', use_raw=True)
    
    #sc.tl.filter_rank_genes_groups(adata, min_in_group_fraction=0, min_fold_change=1, max_out_group_fraction=0)
    
    clustMarkers = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    hpcsLogfc = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
    try:
        hpcsMarkers.append(clustMarkers.loc[:,"hpcs"].values)
        hpcsLogfcs.append(hpcsLogfc.loc[:,"hpcs"].values)
        allHpcsList = np.append(allHpcsList,clustMarkers.loc[:,"hpcs"].values)
    except:
        hpcsMarkers.append(np.array([]))
        hpcsLogfcs.append(np.array([]))

    #try:
    #    sc.pl.umap(adata, color=['SLC4A11', 'ITGA2', 'CLDN4', 'THBD', 'PROCR', 'PAR1', 'PLAUR', 'F2R']+["gmmClass", "kacScore", "HPCSScore","nfkbScore"], ncols=4, cmap="bwr")
    #except:
    sc.pl.umap(adata, color=scoreNames+["gmmClass", "kacScore", "HPCSScore","nfkbScore"], ncols=4, cmap="bwr")
    #hpcsNFKb = adata[adata.obs.gmmClass=="hpcs"].obs["nfkbScore"]
    #otherNFKb = adata[np.logical_not(adata.obs.gmmClass=="hpcs")].obs["nfkbScore"]

    # Check variance (if ratio > 4:1, variances are different)
    #variance_ratio = np.var(hpcsNFKb) / np.var(otherNFKb)
    #print(f"Variance ratio: {variance_ratio:.4f}")

    #t_statistic, p_value = stats.ttest_ind(hpcsNFKb, otherNFKb, equal_var=False, alternative="greater")
    #u_statistic, p_value = stats.mannwhitneyu(hpcsNFKb, otherNFKb, alternative="greater")
    #print(f"\t pval:{p_value}")
    #adata.obs["hpcs"] = ["hpcs" if h else "other" for h in adata.obs.gmmClass=="hpcs"]
    
    #sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
    #sc.pl.violin(adata, keys="nfkbScore", groupby="hpcs",  inner = "box", stripplot=False)
    #sc.pl.violin(adata, keys="kacScore", groupby="hpcs",  inner = "box", stripplot=False)
    #sc.pl.umap(adata, color=scoreNames+["leiden","cellState","decoupler","gmmClass","nfkbScore","kacScore"], ncols=4, cmap="bwr")

    
hpcsMarkers

In [ ]:
hpcsMarkers

In [ ]:
hpcsMfc1 = pd.DataFrame(hpcsMarkers,index=[name.split(".")[0] for name in luadFiles]).T[:200]
hpcsMfc1

In [ ]:
hpcsMfc1logs = pd.DataFrame(hpcsLogfcs,index=[name.split(".")[0] for name in luadFiles]).T[:200]
hpcsMfc1logs

In [ ]:
hpcsM

In [ ]:
hpcsMfc1

In [ ]:
hpcsMfc1#.iloc[:,:9:]

In [ ]:
allHpcs = np.array([])
for dataset in hpcsM.columns:#.iloc[:,:9].columns:
    allHpcs = np.append(allHpcs,hpcsM.loc[:,dataset])
    
allHpcsCount = Counter(allHpcs)
print(allHpcsCount.most_common()[:20])
topOverGenes = [gene for gene,_ in allHpcsCount.most_common()[:20]]

In [ ]:
topOverGenes

In [ ]:
topOverGenes

In [ ]:
allHpcsCount = Counter(allHpcs)
print(allHpcsCount.most_common()[:20])
topOverGenes = [gene for gene,_ in allHpcsCount.most_common()[:20]]

In [ ]:
for i, luadf in enumerate(luadFiles):
    print(luadf)
    adata = sc.read_h5ad(writeDir+luadf)
    sc.pl.umap(adata, color=topOverGenes+["hpcsScore"], ncols=4)

In [ ]:
for i, luadf in enumerate(luadFiles):
    print(luadf)
    adata = sc.read_h5ad(writeDir+luadf)
    sc.pl.umap(adata, color=['SLC4A11', 'ITGA2', 'CLDN4', 'THBD', 'PROCR', 'PLAUR', 'F2R']+["kacScore", "hpcsScore"], ncols=4)

In [ ]:
hpcsMarkers = []
allHpcsList = np.array([])

n_rows, n_cols = 3, 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 8))    


for i, luadf in enumerate(luadFiles):
    #= luadFiles[i]
    #print(luadf)
    adata = sc.read(writeDir+luadf)
    sc.tl.leiden(adata, resolution=0.3)
    scoreNames, newBClabel, ogLabelScoreMed = scH.scoreAndLabel(adata, sigGenes, labelOfSigGenes= sigGenes.columns, ogLabel="leiden", translate=False)
    sc.tl.score_genes(adata, nfkb,score_name='nfkbScore')

    
    gmmClass = gmmScoreGeneSig(adata.obs[scoreNames],show=False)
    adata.obs["gmmClass"] = gmmClass

    hpcsNFKb = adata[adata.obs.gmmClass=="HPCS"].obs["nfkbScore"]
    otherNFKb = adata[np.logical_not(adata.obs.gmmClass=="HPCS")].obs["nfkbScore"]

    # Check variance (if ratio > 4:1, variances are different)
    variance_ratio = np.var(hpcsNFKb) / np.var(otherNFKb)
    print(f"Variance ratio: {variance_ratio:.4f}")

    #t_statistic, p_value = stats.ttest_ind(hpcsNFKb, otherNFKb, equal_var=False, alternative="greater")
    u_statistic, p_value = stats.mannwhitneyu(hpcsNFKb, otherNFKb, alternative="greater")
    print(f"\t pval:{p_value}")
    adata.obs["hpcs"] = ["HPCS" if h else "other" for h in adata.obs.gmmClass=="HPCS"]
    
    #sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
    sc.pl.violin(adata, keys="nfkbScore", groupby="hpcs",  inner = "box", stripplot=False, ax=axes[i//5,i%5], show=False)
    
    axes[i//5,i%5].set_title(f'{luadf[4:][:-5]} pval={p_value:.3f}')
    #axes[i//5,i%5].set_ylabel('nfkbScore')
    
plt.tight_layout()

plt.show()

In [ ]:
for luadf in luadFiles:
    print(luadf)
    adata = sc.read_h5ad(writeDir+luadf)

    sc.pl.umap(adata, color=["NKX2-1","HMGA2","CLDN4","ITGA2","PROCR","F2R","PLAUR","SLC4A11","SOX2","SOX9","VIM","EPCAM"], ncols=4, cmap="Reds")
    


In [ ]:
hpcsM = pd.DataFrame(hpcsMarkers,index=[name.split(".")[0] for name in luadFiles]).T[:200]
hpcsM

In [ ]:
allHpcs = np.array([])
for dataset in hpcsM:
    allHpcs = np.append(allHpcs,hpcsM.loc[:,dataset])
    
len(allHpcs)

In [ ]:
allHpcsCount = Counter(allHpcs)
allHpcsCount

In [ ]:
"TNF" in list(sigGenes["hpcs"])

In [ ]:
for luadf in luadFiles:
    print(luadf)
    adata = sc.read(writeDir+luadf)
    sc.pl.umap(adata, color=topExpHpcs[:10]+["hpcsScore","decoupler"], ncols=4, cmap="Reds")


In [ ]:
dict(allHpcsCount.most_common()[1:20])

In [ ]:
#made with help from chatGPT
import matplotlib.pyplot as plt

# Sample data (names and frequencies)
data = dict(allHpcsCount.most_common()[1:50])

# Sort the data based on frequencies
sorted_data = sorted(data.items(), key=lambda x: x[1], reverse=True)

# Extract names and frequencies
names, frequencies = zip(*sorted_data)

# Plot the elbow plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(frequencies) + 1), frequencies, marker='.', linestyle='')

# Annotate points with names
for i, (name, freq) in enumerate(sorted_data):
    plt.annotate(name, (i + 1, freq), textcoords="offset points", xytext=(0,-50), ha='center', rotation=90)

plt.title('HPCS Genes and Their Frequency')
plt.xlabel('Rank')
plt.ylabel('Frequency')
plt.xticks(range(1, len(names) + 1), [])
plt.ylim(0, max(frequencies) * 1.1)  # Set y-axis limit to start from 0
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
topExpHpcs = [gene for gene,val in allHpcsCount.most_common()[1:50]]
topExpHpcs

In [ ]:
set(sigGenes["hpcs"]).intersection(set(topExpHpcs))